In [12]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import math

dns_flows_File = '../../sgrover/1hour-night/split_hour_00002_20170124015900.csv'

sql_sc = SQLContext(sc)

pandas_df = pd.read_csv(dns_flows_File, sep="|", error_bad_lines=False,
               names=['time', 'srcip', 'dstip', 'dnsquery']) 
# pandas_df = pd.read_csv('file.csv', names = ['column 1','column 2']) # if no header
s_rdd = sql_sc.createDataFrame(pandas_df).rdd

In [13]:
flows_all = s_rdd.collect()
print flows_all[:5]

[Row(time=u'0.004084000', srcip=u'10.9.186.52', dstip=u'10.8.0.5', dnsquery=u'www.youtube.com'), Row(time=u'0.004886000', srcip=u'10.9.186.52', dstip=u'10.8.0.6', dnsquery=u'www.youtube.com'), Row(time=u'0.007861000', srcip=u'10.9.231.134', dstip=u'10.8.0.5', dnsquery=u'instagram.com'), Row(time=u'0.010363000', srcip=u'10.9.186.52', dstip=u'10.8.0.5', dnsquery=u'youtube-ui.l.google.com'), Row(time=u'0.026029000', srcip=u'10.9.185.235', dstip=u'10.8.0.5', dnsquery=u'guzzoni.apple.com')]


In [17]:
src_2_unique_queries = (s_rdd.map(lambda s: ((s[1],s[3]),1))
                       .reduceByKey(lambda x,y: x+y)
                       .map(lambda s: (s[0][0],1))
                       .reduceByKey(lambda x,y: x+y)
                      )
print src_2_unique_queries.take(5)

[(u'10.9.120.227', 13), (u'10.8.147.102', 1), (u'140.180.219.162', 5), (u'10.9.42.157', 17), (u'10.9.131.13', 6)]


In [18]:
src_2_total_queries = (s_rdd.map(lambda s: (s[1],1))
                       .reduceByKey(lambda x,y: x+y)
                      )
print src_2_total_queries.take(5)

[(u'10.9.120.227', 25), (u'10.9.162.26', 35), (u'140.180.219.162', 14), (u'10.9.131.13', 7), (u'10.9.29.134', 30)]


In [53]:
def add_domain(domain):
    n_levels = 3
    tmp = domain.split('.')
    out = []
    for level in range(1, 1+n_levels):
        out.append('.'.join(tmp[-level:]))
    return out
            
    
add_domain('youtube.com')

augmented_data = (s_rdd
                  # add different level domains as columns
                  .map(lambda s: tuple([str(x) for x in s]+add_domain(str(s[3]))))
                  # time in seconds
                  .map(lambda s: tuple([int(float(s[0]))]+[str(x) for x in s[1:]]))
                 )

print augmented_data.take(500)

[(0, '10.9.186.52', '10.8.0.5', 'www.youtube.com', 'com', 'youtube.com', 'www.youtube.com'), (0, '10.9.186.52', '10.8.0.6', 'www.youtube.com', 'com', 'youtube.com', 'www.youtube.com'), (0, '10.9.231.134', '10.8.0.5', 'instagram.com', 'com', 'instagram.com', 'instagram.com'), (0, '10.9.186.52', '10.8.0.5', 'youtube-ui.l.google.com', 'com', 'google.com', 'l.google.com'), (0, '10.9.185.235', '10.8.0.5', 'guzzoni.apple.com', 'com', 'apple.com', 'guzzoni.apple.com'), (0, '10.9.114.94', '10.8.0.5', 'm.wsj.net', 'net', 'wsj.net', 'm.wsj.net'), (0, '140.180.220.103', '128.112.129.209', '131.208.107.116.in-addr.arpa', 'arpa', 'in-addr.arpa', '116.in-addr.arpa'), (0, '10.8.85.202', '10.8.0.5', 'developers.kakao.com', 'com', 'kakao.com', 'developers.kakao.com'), (0, '10.9.171.199', '10.8.0.5', 'mobile-feedlocator.nytimes.com', 'com', 'nytimes.com', 'mobile-feedlocator.nytimes.com'), (0, '10.9.97.208', '10.8.0.5', 'www.qq.com', 'com', 'qq.com', 'www.qq.com'), (0, '140.180.220.25', '128.112.128.1',